In [1]:
import pandas as pd

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys  
stdout=sys.stdout
reload(sys)  
sys.setdefaultencoding('utf8')
#need this line, otherwise print output not printed in notebook
sys.stdout = stdout

In [3]:
authors=[]
def check_files():
    l=glob.glob(basePath+'*.txt')
    for filepath in l:
        print "processing "+filepath
        try:
            f=open(filepath)
            lines=f.readlines()
        except Exception as e:
            print "something went wrong with the file, ignoring it"
            print e
            continue
        
        title_line=lines[0][preamble_len:]
        ofp=title_line.find('of')
        if ofp >=0:
            title_line=title_line[ofp+3:title_line.find('\n')]
            
        title=title_line.split(',')[0]
        
        author_line=lines[1]
        aut_indx=author_line.find('by')
        
        if aut_indx >=0:
            author=author_line[aut_indx+3:author_line.find('\n')]
        else:
            #second try with the first line
            aut_indx=title_line.find('by')
            if aut_indx>0:
                author=title_line[aut_indx+3:title_line.find('\n')]
            else:     
                print ("cannot id author, ignoriing text\n")
                print "author line"+author_line
                print "title line "+ title_line
                continue
    
        print "author identified as "+author
        print "title identified as "+title
        authors.append(author)
        
    

In [4]:
preamble_len=len("The Project Gutenberg EBook")
nLinesToIgnore=100
def openFileWithIds(filepath):
    try:
        f=open(filepath)
        lines=f.readlines()
    except Exception as e:
        print "something went wrong with the file, ignoring it"
        print e
        return "unknown","unknown",""

#     ### identify title ####
#     title_line=lines[0][preamble_len:]
#     ofp=title_line.find('of')
#     if ofp >=0:
#         title_line=title_line[ofp+3:title_line.find('\n')]

#     title=title_line.split(',')[0]

#     ### identify author ###
#     author_line=lines[1]
#     aut_indx=author_line.find('by')

#     if aut_indx >=0:
#         author=author_line[aut_indx+3:author_line.find('\n')]
#     else:
#         #second try with the first line
#         aut_indx=title_line.find('by')
#         if aut_indx>0:
#             author=title_line[aut_indx+3:title_line.find('\n')]
#         else:     
#             print ("cannot id author, ignoriing text\n")
#             print "author line"+author_line
#             print "title line "+ title_line
#             return "unknown","unknown",""
    
    complete_text="".join(lines)
    ########## title search #################
    titid=complete_text.find('Title: ')
    if titid>0:
        title=complete_text[titid+7:complete_text.find('\n',titid)].strip()
    else:
        title_line=lines[0][preamble_len:]
        ofp=title_line.find('of')
        if ofp >=0:
            title=title_line[ofp+3:title_line.find('\n')].strip()
        else:
            title="unknown"
    
    ########## author search #######################
    autid=complete_text.find('Author: ')
    if autid>0:
        author=complete_text[autid+8:complete_text.find('\n',autid)].strip()
    else:
        author_line=lines[1]
        aut_indx=author_line.find('by')
        if aut_indx >=0:
            author=author_line[aut_indx+3:author_line.find('\n')].strip()
        else:
            author="unknown"
        
    ##########decode text#####################
    encid=complete_text.find('encoding')
    
    if encid>=0:
        encoding=complete_text[encid+10:encid+30].strip()
        encoding=encoding[0:encoding.find('\n')]
    else:
        #attempt standard encoding
        encoding='UTF-8'
    
    full_text="".join(lines[nLinesToIgnore:])
    if encoding=='ISO-8859-1':
        full_text=full_text.decode('ISO-8859-1').encode('utf-8')
    elif encoding in ['Latin-1','ISO Latin-1','Latin1','ISO-Latin-1']:
        full_text=full_text.decode('Latin-1').encode('utf-8')
    elif encoding in ['UTF-8','ASCII','ASCII (with a few I','ASCII, with a few I']:
        full_text=full_text.decode('utf-8','ignore').encode("utf-8")
        #many of the text have errouns ascii or utf-8 encoding.
        #remove all erronous symbols and print
    else:
        print "cannot do decoding, running with standard"
        full_text=full_text.decode('utf-8','ignore').encode("utf-8")
    
    f.close()
    
    return author,title,full_text
    

Load and process the Gutenberg input data.
Open a random choice of books, open them. Author seems always noted in the second line, after the word "by"

In [4]:
import glob
import random
import nltk
basePath='C:/Users/david/Code/various_ml/spooky_author_id/gutenberg_data2/'

#df=pd.DataFrame(columns=['author','title','text'])

datalist=[]
def prepGutenBergData(nTextSamples,nSentenceSamples,randomSeed): 
    l=glob.glob(basePath+'*.txt')
    random.seed(randomSeed)
    nSuccess=0
    
    while nSuccess<nTextSamples:
        texts = random.sample(l, nTextSamples-nSuccess)
        for filepath in texts:
            print "processing "+filepath
            author,title,full_text=openFileWithIds(filepath)

            try:
                sentences=nltk.sent_tokenize(full_text)
            except UnicodeDecodeError as e:
                print "something went wrong with decoding besides best attempts"
                print e
                continue

            try:
                sent_choice=random.sample(sentences,nSentenceSamples)
            except ValueError as e:
                print(e)
                continue

            for choice in sent_choice:
                datalist.append({'author':author,'title':title,'text': choice})

            print('\n')
            nSuccess+=1
    return datalist,nSuccess

In [ ]:
dl,nDataPoints=prepGutenBergData(10,50,59)
df=pd.DataFrame(data=dl)
print("nDataPoints: {}".format(nDataPoints))

In [5]:
import string
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
#preprocess the text
# => stopwords removal
# => punctuation removal
# => stemming

def preProcessText(text):
    text=text.lower()
    text=text.translate(string.maketrans('',''),string.punctuation)
    text_list=word_tokenize(text)
    text_list=[w for w in text_list if not w in stop_words]
    final_list=[]
    for w in text_list:
        try:
            w.decode('utf-8')
            final_list.append(w)
        #some words seem to be not in the encoding mentioned in the files. Ignore those
        except UnicodeError:
            continue
    return final_list

In [7]:
#build dictionary
import glob
import random
import nltk
basePath='C:/Users/david/Code/various_ml/spooky_author_id/gutenberg_dickens/'

all_texts=[]
l=glob.glob(basePath+'*.txt')
for filepath in l:
    print ("processing filepath "+filepath)
    author,title,full_text=openFileWithIds(filepath)
    print "author is "+author
    print "title is " + title
    print '\n'
    all_texts.append(preProcessText(full_text))

processing filepath C:/Users/david/Code/various_ml/spooky_author_id/gutenberg_dickens\1400-8.txt


NameError: name 'openFileWithIds' is not defined

In [8]:
from gensim import corpora
dictionary = corpora.Dictionary(all_texts)
dictionary.save('./dickens.dict')  # store the dictionary, for future reference

In [9]:
import glob
import random
import nltk
basePath='C:/Users/david/Code/various_ml/spooky_author_id/gutenberg_dickens/'
randomSeed=100
dictionary.load('./dickens.dict')
class Corpus(object):
     def __iter__(self):
        l=glob.glob(basePath+'*.txt')
        random.seed(randomSeed)
        for filepath in l:
            print ("processing filepath "+filepath)
            author,title,full_text=openFileWithIds(filepath)
            print "author is "+author
            print "title is " + title
            print '\n'
            yield dictionary.doc2bow(preProcessText(full_text))

In [12]:
from gensim import models
corpus=Corpus()
model = models.LdaModel(corpus, id2word=dictionary, num_topics=100)

processing filepath C:/Users/david/Code/various_ml/spooky_author_id/gutenberg_dickens\1400-8.txt
author is Charles Dickens
title is Great Expectations


processing filepath C:/Users/david/Code/various_ml/spooky_author_id/gutenberg_dickens\1406.txt
cannot do decoding, running with standard
author is Charles Dickens
title is The Perils of Certain English Prisoners


processing filepath C:/Users/david/Code/various_ml/spooky_author_id/gutenberg_dickens\1407.txt
cannot do decoding, running with standard
author is Charles Dickens
title is A Message from the Sea


processing filepath C:/Users/david/Code/various_ml/spooky_author_id/gutenberg_dickens\1415.txt
cannot do decoding, running with standard
author is Charles Dickens
title is Doctor Marigold


processing filepath C:/Users/david/Code/various_ml/spooky_author_id/gutenberg_dickens\1416.txt
cannot do decoding, running with standard
author is Charles Dickens
title is Mrs. Lirriper's Lodgings


processing filepath C:/Users/david/Code/various

processing filepath C:/Users/david/Code/various_ml/spooky_author_id/gutenberg_dickens\17879-8.txt
author is Charles Dickens
title is Barnab� Rudge, Tome I


processing filepath C:/Users/david/Code/various_ml/spooky_author_id/gutenberg_dickens\17880-8.txt
author is Charles Dickens
title is Barnab� Rudge, Tome II


processing filepath C:/Users/david/Code/various_ml/spooky_author_id/gutenberg_dickens\2324.txt
cannot do decoding, running with standard
author is Charles Dickens
title is A House to Let


processing filepath C:/Users/david/Code/various_ml/spooky_author_id/gutenberg_dickens\25985.txt
cannot do decoding, running with standard
author is Percy Fitzgerald
title is Bardell v. Pickwick


processing filepath C:/Users/david/Code/various_ml/spooky_author_id/gutenberg_dickens\46-8.txt
author is Charles Dickens
title is A Christmas Carol


processing filepath C:/Users/david/Code/various_ml/spooky_author_id/gutenberg_dickens\699.txt
cannot do decoding, running with standard
author is Char

In [18]:
main_data=pd.read_csv('ted_main.csv')
transscripts=pd.read_csv('transcripts.csv')

In [22]:
transscripts.head(0)

,transcript,url


In [25]:
df=main_data.set_index('url').join(transscripts.set_index('url'))

In [49]:
global_list=[]
for r in df.iterrows():
    l=r[1]['tags'].split(',')
    for w in l:
        w2=w.translate(string.maketrans('',''),string.punctuation)
        global_list.append(w2)

In [51]:
tags=pd.DataFrame(data=global_list)

In [53]:
tags.describe()

,0
count,19167
unique,594
top,technology
freq,726


In [32]:
transscripts['url'].describe()

count                                                  2467
unique                                                 2464
top       https://www.ted.com/talks/brene_brown_listenin...
freq                                                      2
Name: url, dtype: object

In [ ]:
train_data=pd.read_csv('train.csv')

In [28]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [ ]:
import string
data=pd.read_csv('./train.csv')
data['lower_case']=data['text'].apply(lambda x: x.lower())
#python 3.
#data['no_punct']=data['lower_case'].apply(lambda x: x.translate(string.maketrans('', '', string.punctuation)))
#python 2.7
data['no_punct']=data['lower_case'].apply(lambda x: x.translate(string.maketrans('', ''), string.punctuation))
data['tokenized']=data['no_punct'].apply(word_tokenize)
data['tokenized_no_stop_words']=data['tokenized'].apply(lambda x: [w for w in x if not w in stop_words])

In [ ]:
test_data=pd.read_csv('test.csv')
count_vector = CountVectorizer()

In [63]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
from sklearn.cross_validation import train_test_split
data=df

X_train, X_test, y_train, y_test = train_test_split(data['text'], data['author'], random_state=1)

print('Number of rows in the total set: {}'.format(data.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 25000
Number of rows in the training set: 18750
Number of rows in the test set: 6250


In [64]:
count_vector = CountVectorizer()
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)

In [65]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()
clf.fit(training_data,y_train)
pred_prop=clf.predict_proba(testing_data)
pred=clf.predict(testing_data)

In [66]:
from sklearn.metrics import log_loss,accuracy_score
print "logloss {}".format(log_loss(y_test,pred_prop))
print "accuacy {}".format(accuracy_score(y_test,pred))

logloss 8.26492656003
accuacy 0.26432


In [71]:
df['author'].astype('category').describe()

count       25000
unique         45
top       Various
freq         3000
Name: author, dtype: object

In [72]:
df

,author,text,title
0,Charles Reade,"Master is in Yorkshire, and that there crocodile, Peggy, she is turned\naway--for sauce--and a good riddance of bad rubbish: Miss is in the\nparlour.""",Hard Cash
1,Charles Reade,"""Oh, thank you, madam.""",Hard Cash
2,Charles Reade,"During the fight, shriek upon shriek issued from the drawing-room.",Hard Cash
3,Charles Reade,He could not keep his hands off a dusty white.,Hard Cash
4,Charles Reade,*Perhaps even this faint attempt at self-analysis was due to the\ninfluence of Dr. Whately.,Hard Cash
5,Charles Reade,Mamma!,Hard Cash
6,Charles Reade,"Only by some half vegetable instinct this darkened\nman was turning towards the morning sun, and staring it full in the face.",Hard Cash
7,Charles Reade,"He added, with\nnautical simplicity and love of cleanliness, ""However, the deck will be\ncleaned and holy-stoned to-morrow, long before you turn out.""",Hard Cash
8,Charles Reade,"I'll bring it you down.""",Hard Cash
9,Charles Reade,"""Ay,"" said Edward, ""but after all he did not sign it.""",Hard Cash


In [73]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [78]:
>>> from gensim import corpora
>>>
>>> documents = ["Human machine interface for lab abc computer applications",
>>>              "A survey of user opinion of computer system response time",
>>>              "The EPS user interface management system",
>>>              "System and human system engineering testing of EPS",
>>>              "Relation of user perceived response time to error measurement",
>>>              "The generation of random binary unordered trees",
>>>              "The intersection graph of paths in trees",
>>>              "Graph minors IV Widths of trees and well quasi ordering",
>>>              "Graph minors A survey"]

In [75]:
corpus

[[(0, 1.0), (1, 1.0), (2, 1.0)],
 [(2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (8, 1.0)],
 [(1, 1.0), (3, 1.0), (4, 1.0), (7, 1.0)],
 [(0, 1.0), (4, 2.0), (7, 1.0)],
 [(3, 1.0), (5, 1.0), (6, 1.0)],
 [(9, 1.0)],
 [(9, 1.0), (10, 1.0)],
 [(9, 1.0), (10, 1.0), (11, 1.0)],
 [(8, 1.0), (10, 1.0), (11, 1.0)]]

In [79]:
>>> # remove common words and tokenize
>>> stoplist = set('for a of the and to in'.split())
>>> texts = [[word for word in document.lower().split() if word not in stoplist]
>>>          for document in documents]
>>>
>>> # remove words that appear only once
>>> from collections import defaultdict
>>> frequency = defaultdict(int)
>>> for text in texts:
>>>     for token in text:
>>>         frequency[token] += 1
>>>
>>> texts = [[token for token in text if frequency[token] > 1]
>>>          for text in texts]
>>>
>>> from pprint import pprint  # pretty-printer
>>> pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [77]:
>>> vec = [(0, 1), (4, 1)]
>>> print(tfidf[vec])
[(0, 0.8075244), (4, 0.5898342)]

[(0, 0.80752440244407231), (4, 0.58983416267400446)]


[(0, 0.8075244), (4, 0.5898342)]

In [ ]:
from shutil import copyfile
import os
all_files=[(x[0],x[2]) for x in os.walk('D:/')]


for t in all_files:
    fnames=t[1]
    for f in fnames:
        totPath=t[0]+"/"+f
        totPath=os.path.join("",totPath)
        if totPath[-3:]=='ZIP':
            print totPath
            copyfile(totPath, 'C:/Users/david/Code/various_ml/spooky_author_id/gutenberg_data2/'+f) 

In [57]:
import rdflib

In [60]:
g = rdflib.Graph()
result = g.parse("C:/Users/david/Downloads/gutenberg_repo/cache/epub/4/pg4.rdf",format='xml')

print("graph has %s statements." % len(g))
# prints graph has 79 statements.

for subj, pred, obj in g:
    print subj
    print pred
    print obj
    print "\n"
    if (subj, pred, obj) not in g:
        raise Exception("It better be!")

s = g.serialize(format='n3')

graph has 144 statements.
http://www.gutenberg.org/files/4/4-h.zip
http://purl.org/dc/terms/format
N6449420190c0431da2d700f57d05a506


http://www.gutenberg.org/ebooks/4
http://purl.org/dc/terms/title
Lincoln's Gettysburg Address
Given November 19, 1863 on the battlefield near Gettysburg, Pennsylvania, USA


http://en.wikipedia.org/wiki/Abraham_Lincoln
http://purl.org/dc/terms/description
Wikipedia


http://www.gutenberg.org/ebooks/4
http://www.gutenberg.org/2009/pgterms/downloads
58


http://www.gutenberg.org/files/4/4-h/4-h.htm
http://purl.org/dc/terms/modified
2015-04-01T02:12:52


http://www.gutenberg.org/ebooks/4.epub.noimages
http://purl.org/dc/terms/modified
2017-10-01T01:01:15.170295


http://www.gutenberg.org/ebooks/4
http://purl.org/dc/terms/hasFormat
http://www.gutenberg.org/ebooks/4.rdf


http://www.gutenberg.org/ebooks/4
http://purl.org/dc/terms/hasFormat
http://www.gutenberg.org/files/4/4.zip


http://www.gutenberg.org/ebooks/4.rdf
http://purl.org/dc/terms/modified
2018-02

In [3]:
from sklearn.metrics import normalized_mutual_info_score

In [15]:
A=['A','A','B','C','A']
B=['D','D','F','G','D']

In [16]:
normalized_mutual_info_score(A,B)

1.0